# 06.03.2025

Задание 1
При старте приложения запускаются три потока.
Первый поток заполняет список случайными числами.
Два других потока ожидают заполнения. Когда список
заполнен оба потока запускаются. Первый поток находит
сумму элементов списка, второй поток среднеарифметическое значение в списке. Полученный список, сумма и
среднеарифметическое выводятся на экран. 


In [2]:
import threading
import random
import time

# Глобальные переменные для синхронизации потоков
list_of_numbers = []
event = threading.Event()

def fill_list_with_random_numbers():
    global list_of_numbers
    print("Заполнение списка случайными числами...")
    for _ in range(10):  # добавляем 10 случайных чисел
        list_of_numbers.append(random.randint(1, 100))
    print("Список заполнен.")
    event.set()  # устанавливаем флаг, сигнализирующий о завершении заполнения списка

def calculate_sum():
    event.wait()  # ждем, пока список будет заполнен
    print("Вычисляю сумму элементов списка...")
    total_sum = sum(list_of_numbers)
    print(f"Сумма элементов списка: {total_sum}")

def calculate_average():
    event.wait()  # ждем, пока список будет заполнен
    print("Вычисляю среднее арифметическое...")
    average = sum(list_of_numbers) / len(list_of_numbers)
    print(f"Среднее арифметическое: {average:.2f}")

def main():
    # Запускаем потоки
    filling_thread = threading.Thread(target=fill_list_with_random_numbers)
    sum_thread = threading.Thread(target=calculate_sum)
    avg_thread = threading.Thread(target=calculate_average)

    # Запускаем первый поток для заполнения списка
    filling_thread.start()

    # Запускаем остальные потоки
    sum_thread.start()
    avg_thread.start()

    # Ожидаем завершения всех потоков
    filling_thread.join()
    sum_thread.join()
    avg_thread.join()

    # Выводим заполненный список
    print(f"Полученный список: {list_of_numbers}")

if __name__ == "__main__":
    main()

Заполнение списка случайными числами...
Список заполнен.
Вычисляю сумму элементов списка...
Сумма элементов списка: 519
Вычисляю среднее арифметическое...
Среднее арифметическое: 51.90
Полученный список: [50, 63, 24, 77, 48, 61, 29, 91, 43, 33]


Задание 2
Пользователь с клавиатуры вводит путь к файлу.
После чего запускаются три потока. Первый поток заполняет файл случайными числами. Два других потока
ожидают заполнения. Когда файл заполнен оба потока
стартуют. Первый поток находит все простые числа, второй поток факториал каждого числа в файле. Результаты
поиска каждый поток должен записать в новый файл. На
экран необходимо отобразить статистику выполненных
операций.

In [1]:
import threading
from random import randint
import math

# Функция для проверки простоты числа
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

# Поток для записи случайных чисел в файл
def write_random_numbers(file_path, event):
    with open(file_path, 'w') as f:
        # Записываем 100 случайных чисел (можно изменить количество)
        for _ in range(100):
            num = str(randint(1, 100))
            f.write(num + '\n')
    print("Запись завершена")
    event.set()  # Сигнализируем другим потокам о завершении записи

# Поток для нахождения всех простых чисел в файле
def find_primes(file_path, prime_file_path, event):
    event.wait()  # Ожидаем завершения записи
    primes = []
    with open(file_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]
    
    for number in numbers:
        if is_prime(number):
            primes.append(str(number))
            
    with open(prime_file_path, 'w') as f:
        f.write('\n'.join(primes))
        
    print(f"Простые числа найдены: {len(primes)}")

# Поток для вычисления факториала каждого числа в файле
def calculate_factorials(file_path, factorial_file_path, event):
    event.wait()  # Ожидаем завершения записи
    factorials = []
    with open(file_path, 'r') as f:
        numbers = [int(line.strip()) for line in f]
    
    for number in numbers:
        factorial = math.factorial(number)
        factorials.append(str(factorial))
        
    with open(factorial_file_path, 'w') as f:
        f.write('\n'.join(factorials))
        
    print(f"Факториалы вычислены: {len(factorials)}")

if __name__ == "__main__":
    file_path = input("Введите путь к файлу: ")
    prime_file_path = "primes.txt"
    factorial_file_path = "factorials.txt"
    
    event = threading.Event()
    
    threads = [
        threading.Thread(target=write_random_numbers, args=(file_path, event)),
        threading.Thread(target=find_primes, args=(file_path, prime_file_path, event)),
        threading.Thread(target=calculate_factorials, args=(file_path, factorial_file_path, event))
    ]
    
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()

Запись завершена
Простые числа найдены: 23
Факториалы вычислены: 100


Задание 3
Пользователь с клавиатуры вводит путь к существующей директории и к новой директории. После чего
запускается поток, который должен скопировать содержимое директории в новое место. Необходимо сохранить
структуру директории. На экран необходимо отобразить
статистику выполненных операций.

In [2]:
import os
import shutil
import threading

def copy_directory(source_dir, destination_dir, event):
    try:
        shutil.copytree(source_dir, destination_dir)
        print(f"Копирование завершилось успешно.\nИсходная директория: {source_dir}\nНовая директория: {destination_dir}")
        event.set()
    except Exception as e:
        print(f"Ошибка при копировании: {str(e)}")

if __name__ == "__main__":
    source_dir = input("Введите путь к исходной директории: ").strip()
    destination_dir = input("Введите путь к новой директории: ").strip()
    
    event = threading.Event()
    
    thread = threading.Thread(target=copy_directory, args=(source_dir, destination_dir, event))
    thread.start()
    
    thread.join()
    print("\nОперация выполнена.")

Ошибка при копировании: [WinError 183] Невозможно создать файл, так как он уже существует: 'C:\\Users\\Jemi\\Documents\\Dz- Test'

Операция выполнена.


Задание 4
Пользователь склавиатурывводитпутьксуществующей
директории и слово для поиска. После чего запускаются
два потока. Первый должен найти файлы, содержащие
искомое слово и слить их содержимое в один файл. Второй поток ожидает завершения работы первого потока.
После чего проводит вырезание всех запрещенных слов
(список этих слов нужно считать из файла с запрещенными словами) из полученного файла. На экран необходимо
отобразить статистику выполненных операций.

In [ ]:
import os
from threading import Thread
import re

def find_and_merge_files(directory_path, search_word):
    """ Функция находит все файлы в директории и её подпапках, содержащие заданное слово, и объединяет их содержимое в один файл. """
    merged_file = 'merged.txt'
    
    # Поиск файлов и объединение содержимого
    with open(merged_file, 'w', encoding='utf-8') as output_file:
        for root, dirs, files in os.walk(directory_path):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                try:
                    with open(file_path, 'r', encoding='utf-8') as input_file:
                        if search_word.lower() in input_file.read().lower():
                            print(f'Файл {file_path} содержит слово "{search_word}"')
                            output_file.write(input_file.read())
                except UnicodeDecodeError:
                    print(f'Ошибка декодирования файла {file_path}')
    return merged_file

def remove_banned_words(file_path, banned_words):
    """ Функция удаляет запрещённые слова из файла. """
    with open(file_path, 'r+', encoding='utf-8') as file:
        content = file.read()
        
        # Удаление запрещённых слов
        for word in banned_words:
            pattern = r'\b{}\b'.format(re.escape(word))
            content = re.sub(pattern, '', content, flags=re.IGNORECASE)
        
        # Запись очищенного текста обратно в файл
        file.seek(0)
        file.truncate()
        file.write(content)

def main():
    directory_path = input("Введите путь к директории: ")
    search_word = input("Введите слово для поиска: ").strip()
    banned_words_file = input("Введите путь к файлу с запрещёнными словами: ")
    
    # Чтение списка запрещённых слов
    with open(banned_words_file, 'r', encoding='utf-8') as f:
        banned_words = [line.strip() for line in f.readlines()]
    
    # Создание первого потока для поиска и слияния файлов
    thread1 = Thread(target=find_and_merge_files, args=(directory_path, search_word))
    thread1.start()
    
    # Ожидание завершения работы первого потока
    thread1.join()
    
    # Получаем имя объединённого файла
    merged_file = 'merged.txt'
    
    # Создание второго потока для удаления запрещённых слов
    thread2 = Thread(target=remove_banned_words, args=(merged_file, banned_words))
    thread2.start()
    
    # Ожидание завершения работы второго потока
    thread2.join()
    
    # Статистика выполненных операций
    print("\nОперации завершены:")
    print(f"Объединено файлов: {len(os.listdir(directory_path))}")
    print(f"Удалено запрещённых слов: {len(banned_words)}")

if __name__ == "__main__":
    main()